In [ ]:
import numpy as np
import tensorflow as tf
import time
from IPython.display import clear_output

%matplotlib inline

In [ ]:
games_num = 1984694

move_dict = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5, 
             'g':6, 'h':7, 'j':8, 'k':9, 'l':10, 'm':11, 
             'n':12, 'o':13, 'p':14}
letters = ['a', 'b', 'c', 'd', 'e', 'f', 
           'g', 'h', 'j', 'k', 'l', 'm', 
           'n', 'o', 'p']
for i in range(15):
    move_dict[i] = letters[i]

def AllGraph():
    def Conv2D(filters, kernel=5, name=None):
        return tf.layers.Conv2D(filters, kernel, padding='same', name=name)

    def BatchNorm(name=None):
        return tf.keras.layers.BatchNormalization()
    
    p_dict = {}
    r_dict = {}
    
    tf.reset_default_graph()
    
    x = tf.placeholder(tf.float32, shape=[None, 2, 15, 15])
    y_ = tf.placeholder(tf.int64)
    phase = tf.placeholder(tf.bool)
    alpha = tf.placeholder(tf.float32)
    p_dict['x'] = x
    p_dict['y_'] = y_
    p_dict['phase'] = phase
    p_dict['alpha'] = alpha
    r_dict['x'] = x
    r_dict['y_'] = y_
    r_dict['phase'] = phase
    r_dict['alpha'] = alpha
        
    tran = tf.transpose(x, [0, 2, 3, 1])
        
    model = Conv2D(32)(tran)
    model = BatchNorm()(model, training=phase)
    model = tf.nn.relu(model)
        
    model = Conv2D(32)(model)
    model = BatchNorm()(model, training=phase)
    model = tf.nn.relu(model)
        
    model = Conv2D(32)(model)
    model = BatchNorm()(model, training=phase)
    model = tf.nn.relu(model)
        
    model = Conv2D(32)(model)
    model = BatchNorm()(model, training=phase)
    model = tf.nn.relu(model)
        
    model = Conv2D(32)(model)
    model = BatchNorm()(model, training=phase)
    model = tf.nn.relu(model)
        
    model = Conv2D(32)(model)
    model = BatchNorm()(model, training=phase)
    model = tf.nn.relu(model)
        
    model = Conv2D(64)(model)
    model = BatchNorm()(model, training=phase)
    model = tf.nn.relu(model)
        
    model = Conv2D(64)(model)
    model = BatchNorm()(model, training=phase)
    model = tf.nn.relu(model)
        
    out = tf.layers.Flatten()(Conv2D(1, 3)(model))
        
    move = tf.nn.softmax(out)
    p_dict['move'] = move

    check_prediction = tf.equal(tf.argmax(out,1), y_)
    accuracy = tf.reduce_mean(tf.cast(check_prediction, tf.float32))
    p_dict['accuracy'] = accuracy

    loss_function = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_, logits=out))
    p_dict['loss'] = loss_function

    extra_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_ops):
        train_step = tf.train.AdamOptimizer(alpha).minimize(loss_function)
    p_dict['train_step'] = train_step
            
            
    model = Conv2D(32)(tran)
    model = BatchNorm()(model, training=phase)
    model = tf.nn.relu(model)
        
    model = Conv2D(32)(model)
    model = BatchNorm()(model, training=phase)
    model = tf.nn.relu(model)
        
    model = Conv2D(32)(model)
    model = BatchNorm()(model, training=phase)
    model = tf.nn.relu(model)
        
    model = Conv2D(64)(model)
    model = BatchNorm()(model, training=phase)
    model = tf.nn.relu(model)
        
    out = tf.layers.Flatten()(Conv2D(1, 3)(model))
        
    move = tf.nn.softmax(out)
    r_dict['move'] = move

    check_prediction = tf.equal(tf.argmax(out,1), y_)
    accuracy = tf.reduce_mean(tf.cast(check_prediction, tf.float32))
    r_dict['accuracy'] = accuracy

    loss_function = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_, logits=out))
    r_dict['loss'] = loss_function

    extra_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_ops):
        train_step = tf.train.AdamOptimizer(alpha).minimize(loss_function)
    r_dict['train_step'] = train_step
        
    graph = tf.get_default_graph()
    p_dict['graph'] = graph
    r_dict['graph'] = graph
            
    return {'policy': p_dict, 'rollout': r_dict, 'graph': graph}

In [ ]:
def TrainGen(batch_size, left=0, right=9999999999, aug=False, end=False):
    batch_X = np.zeros((0, 2, 15, 15))
    batch_Y = np.zeros((0,))
    global_epoch = 0
    time_begin = time.time()
    while True:
        if global_epoch > 0 and end:
            break
        train_file = open('train_data.renju')
        game_ind = 0
        for game in train_file:
            if game_ind < left:
                game_ind += 1
                continue
            if game_ind == right:
                break
            game_ind += 1
            win = game[0]
            if win != 'b' and win != 'w':
                continue
            game = game.split(' ')[1:]
            board = np.zeros((len(game), 2, 15, 15))
            moves = np.zeros((len(game),))
            if aug and game_ind%2 == 1:
                aug_type = np.random.randint(0, 3)
            player = 0
            for ind, turn in enumerate(game):
                x = move_dict[turn[0]]
                y = int(turn[1:])-1
                if aug and game_ind%2 == 1:
                    if aug_type == 0:
                        x = 14-x
                        y = 14-y
                    elif aug_type == 1:
                        x = 14-x
                    elif aug_type == 2:
                        temp = x
                        x = y
                        y = temp
                if ind != len(game)-1:
                    board[ind+1] = board[ind]
                    board[ind+1][player][x][y] = 1
                moves[ind] = 15*x + y
                if player == 0:
                    player = 1
                else:
                    player = 0
            batch_X = np.append(batch_X, board, axis=0)
            batch_Y = np.append(batch_Y, moves, axis=0)
            if len(batch_X) >= batch_size:
                yield batch_X, batch_Y, time.time()-time_begin, global_epoch, game_ind
                time_begin = time.time()
                batch_X = np.zeros((0, 2, 15, 15))
                batch_Y = np.zeros((0,))
        global_epoch += 1
    if len(batch_X) > 0:
        yield batch_X, batch_Y, time.time()-time_begin, global_epoch-1, game_ind

In [ ]:
def SupervisedTrain(sess, graph_dict, epochs=5, batches_per_epoch=200, 
                    train_games=games_num-1000, 
                    validation_games=1000, batch_size=100, 
                    alph=0.0001, name='model'):
    train_games = games_num-validation_games
    
    x = graph_dict['policy']['x']
    y_ = graph_dict['policy']['y_']
    phase = graph_dict['policy']['phase']
    alpha = graph_dict['policy']['alpha']
    
    accuracy_p = graph_dict['policy']['accuracy']
    loss_p = graph_dict['policy']['loss']
    train_step_p = graph_dict['policy']['train_step']
    
    accuracy_r = graph_dict['rollout']['accuracy']
    loss_r = graph_dict['rollout']['loss']
    train_step_r = graph_dict['rollout']['train_step']
    
    saver = tf.train.Saver()
    saver.save(sess, name, write_meta_graph=False)
    
    log_count = 0
    last_val_acc_p = 0
    last_val_loss_p = float('inf')
    last_val_acc_r = 0
    last_val_loss_r = float('inf')
    train_gen = TrainGen(batch_size, 0, train_games)
    begin_time = time.time()
    train_acc_p = 0
    train_acc_r = 0
    
    def info_log(log_count):
        if log_count == 70:
            clear_output()
            log_count = 0
        else:
            log_count += 1
            return log_count
        log_count += 1
        print('----------------------------------------------------')
        print('Global epoch:', g_epoch)
        print('Local epoch:', epoch+1, '/', epochs)
        print('Batch:', batch_ind, '/', batches_per_epoch)
        print('Epoch time:', (time.time()-epoch_time)/60, '/', (((time.time()-epoch_time)/batch_ind)*batches_per_epoch)/60)
        print('Time for making one batch:', batch_time)
        print('Game:', game_ind, '/', train_games)
        print('Train accuracy (policy):', train_acc_p)
        print('Train loss (policy):', train_loss_p)
        print('Train accuracy (rollout):', train_acc_r)
        print('Train loss (rollout):', train_loss_r)
        print('Valid acc (policy):', last_val_acc_p)
        print('Valid loss (policy):', last_val_loss_p)
        print('Valid acc (rollout):', last_val_acc_r)
        print('Valid loss (rollout):', last_val_loss_r)
        print('----------------------------------------------------')
        return log_count
    
    for epoch in range(epochs):
        batch_ind = 0
        epoch_time = time.time()
        for batch_X, batch_Y, batch_time, g_epoch, game_ind in train_gen:
            info = sess.run([accuracy_p, loss_p, train_step_p, accuracy_r, loss_r, train_step_r], 
                            feed_dict={x: batch_X, y_: batch_Y, phase: True, alpha: alph})
            train_acc_p = info[0]
            train_loss_p = info[1]
            train_acc_r = info[3]
            train_loss_r = info[4]
        
            batch_ind += 1
            log_count = info_log(log_count)
            if batch_ind == batches_per_epoch:
                break
        valid_gen = TrainGen(batch_size, games_num-validation_games, aug=False, end=True)
        valid_ind = 0
        last_val_loss_p = 0
        last_val_acc_p = 0
        last_val_loss_r = 0
        last_val_acc_r = 0
        for valid_X, valid_Y, batch_time, g_epoch, game_ind in valid_gen:
            info = sess.run([accuracy_p, loss_p, accuracy_r, loss_r], feed_dict={x: valid_X, y_: valid_Y, phase: False})
            last_val_acc_p += info[0]
            last_val_loss_p += info[1]
            last_val_acc_r += info[2]
            last_val_loss_r += info[3]
            valid_ind += 1
        last_val_acc_p /= valid_ind
        last_val_loss_p /= valid_ind
        last_val_acc_r /= valid_ind
        last_val_loss_r /= valid_ind
        saver.save(sess, name, global_step=epoch+1, write_meta_graph=False)
        log_count = info_log(log_count)

In [ ]:
graph_dict = AllGraph()

In [ ]:
local_epochs = 35
batches_per_epoch = 8000
validation_games = 3000
batch_size = 100
alph = 0.0001
name = 'checkpoints/allgraph'
with tf.Session(graph=graph_dict['graph']) as sess:
    sess.run(tf.global_variables_initializer())
    try:
        SupervisedTrain(sess, graph_dict, batches_per_epoch=batches_per_epoch, 
                                          validation_games=validation_games,
                                          batch_size=batch_size,
                                          alph=alph,
                                          name=name,
                                          local_epochs=epochs)
        saver = tf.train.Saver()
        saver.save(sess, name+'-fin', write_meta_graph=False)
    except KeyboardInterrupt:
        saver = tf.train.Saver()
        saver.save(sess, name+'-fin', write_meta_graph=False)